In [ ]:
import numpy as np
from bs4 import BeautifulSoup
import requests
import numpy as np 
import pandas as pd
!pip install aiohttp

In [ ]:
linksUnique = pd.read_parquet('allPlayersLink.parquet')['PlayersLink'].unique().tolist()
for i in range(len(linksUnique)):
  linksUnique[i] = 'https://www.transfermarkt.com'+linksUnique[i]


In [ ]:
start = time.time()
test = pd.read_parquet('totalPlayers.parquet',engine='pyarrow')
for i in range(0,9400,100):
  import asyncio
  import aiohttp
  import time
  import nest_asyncio
  nest_asyncio.apply()

  playersLink=[]
  index=0
  playersId , playersName , season_ , date_ , left_,joined_, mv_ , fee_ = [],[],[],[],[],[],[],[]

  async def get(url, session):
      try:
          async with session.get(url=url) as response:
              resp = await response.read()
              id1=url.split('profil/spieler/')[1]
              name=test[test['playersId']==id1]['playersName'].iloc[0]
              pageSoup = BeautifulSoup(resp,'html.parser')
              TransferData = pageSoup.find_all('div',{'class':'box viewport-tracking'})[1]
              season  = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__season'})
              date = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__date'})
              left = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__old-club'})
              joined = TransferData.find_all('div',{'grid__cell grid__cell--center tm-player-transfer-history-grid__new-club'})
              mv = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__market-value'})
              fee = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__fee'})
              for i in range(len(season)):
                  playersId.append(id1)
                  playersName.append(name)
                  try:
                    season_.append(season[i].text.strip())
                  except:
                    season_.append(np.nan)
                  try:
                    date_.append(date[i].text.strip())
                  except:
                    date_.append(np.nan)
                  try:
                    left_.append([left[i].text.strip(),left[i].a.get('href').split('transfers/verein/')[1].split('/')[0]])
                  except:
                    left_.append(np.nan)
                  try:
                    joined_.append([joined[i].text.strip(),joined[i].a.get('href').split('transfers/verein/')[1].split('/')[0]])
                  except:
                    joined_.append(np.nan)
                  try:
                    mv_.append(mv[i].text.strip())
                  except:
                    mv_.append(np.nan)
                  try:
                    fee_.append(fee[i].text.strip())
                  except:
                    fee_.append(np.nan)
      except Exception as e:
          print("Unable to get url {} due to {}.".format(url, e.__class__))
  connector = aiohttp.TCPConnector(limit=20)
      

  async def main(urls):
      async with aiohttp.ClientSession(connector=connector) as session:
          ret = await asyncio.gather(*[get(url, session) for url in urls])
      print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
      if(i==0):
        df = pd.DataFrame({'playersId':playersId,'season':season_ , 'date':date_ , 'left':left_ , 'joined':joined_ ,\
          'mv':mv_ , 'fee':fee_})
      else :
        df1 = pd.DataFrame({'playersId':playersId,'season':season_ , 'date':date_ , 'left':left_ , 'joined':joined_ ,\
          'mv':mv_ , 'fee':fee_})
        df=pd.concat([df,df1],axis=0)



  urls = linksUnique[i:i+100]
  asyncio.run(main(urls))

end = time.time()
print("Took {} seconds to pull {} websites.".format(end - start, len(urls)))

In [ ]:
df.to_parquet('AllplayersTransferFinal.parquet',engine='pyarrow')